In [4]:
from api_keys import weather_api_key
import pandas as pd
from sqlalchemy import create_engine
import requests
import datetime

In [5]:
csv_file = "California_Fire_Incidents.csv"
cal_Fire_Inci_df = pd.read_csv(csv_file)
cal_Fire_Inci_df.head()

,AcresBurned,Active,AdminUnit,AirTankers,ArchiveYear,CalFireIncident,CanonicalUrl,ConditionStatement,ControlStatement,Counties,...,SearchKeywords,Started,Status,StructuresDamaged,StructuresDestroyed,StructuresEvacuated,StructuresThreatened,UniqueId,Updated,WaterTenders
0,257314.0,False,Stanislaus National Forest/Yosemite National Park,NaN,2013,True,/incidents/2013/8/17/rim-fire/,NaN,NaN,Tuolumne,...,"Rim Fire, Stanislaus National Forest, Yosemite...",2013-08-17T15:25:00Z,Finalized,NaN,NaN,NaN,NaN,5fb18d4d-213f-4d83-a179-daaf11939e78,2013-09-06T18:30:00Z,NaN
1,30274.0,False,USFS Angeles National Forest/Los Angeles Count...,NaN,2013,True,/incidents/2013/5/30/powerhouse-fire/,NaN,NaN,Los Angeles,...,"Powerhouse Fire, May 2013, June 2013, Angeles ...",2013-05-30T15:28:00Z,Finalized,NaN,NaN,NaN,NaN,bf37805e-1cc2-4208-9972-753e47874c87,2013-06-08T18:30:00Z,NaN
2,27531.0,False,CAL FIRE Riverside Unit / San Bernardino Natio...,NaN,2013,True,/incidents/2013/7/15/mountain-fire/,NaN,NaN,Riverside,...,"Mountain Fire, July 2013, Highway 243, Highway...",2013-07-15T13:43:00Z,Finalized,NaN,NaN,NaN,NaN,a3149fec-4d48-427c-8b2c-59e8b79d59db,2013-07-30T18:00:00Z,NaN
3,27440.0,False,Tahoe National Forest,NaN,2013,False,/incidents/2013/8/10/american-fire/,NaN,NaN,Placer,...,"American Fire, August 2013, Deadwood Ridge, Fo...",2013-08-10T16:30:00Z,Finalized,NaN,NaN,NaN,NaN,8213f5c7-34fa-403b-a4bc-da2ace6e6625,2013-08-30T08:00:00Z,NaN
4,24251.0,False,Ventura County Fire/CAL FIRE,NaN,2013,True,/incidents/2013/5/2/springs-fire/,Acreage has been reduced based upon more accur...,NaN,Ventura,...,"Springs Fire, May 2013, Highway 101, Camarillo...",2013-05-02T07:01:00Z,Finalized,6.0,10.0,NaN,NaN,46731fb8-3350-4920-bdf7-910ac0eb715c,2013-05-11T06:30:00Z,11.0


In [6]:
year = 2013
for index,row in cal_Fire_Inci_df[['Latitude','Longitude', 'ArchiveYear']].iterrows():
    if row[2] != year:
        year += 1
    print(row[0],row[1])
    print (year)

37.857 -120.086
2013
34.585595 -118.423176
2013
33.7095 -116.72885
2013
39.12 -120.65
2013
0.0 0.0
2013
37.279 -119.318
2013
33.86157 -116.90427
2013
41.32 -123.176
2013
41.035 -123.488
2013
40.042629999999996 -121.85396999999999
2013
40.498332 -122.535496
2013
32.95435 -116.47381000000001
2013
40.190062 -121.595555
2013
34.7861 -118.9411
2013
34.7861 -118.9411
2013
38.8167 -122.8498
2013
34.288877 -116.94131100000001
2013
37.90757 -121.88258200000001
2013
33.121109999999994 -116.52578999999999
2013
39.446267999999996 -121.38236
2013
33.341 -117.3092
2013
36.208 -118.444
2013
34.55048 -119.82428999999999
2013
37.58202 -119.91071000000001
2013
33.62236 -117.40288000000001
2013
33.044579999999996 -116.5209
2013
38.25108 -120.02778
2013
0.0 0.0
2013
35.707 -118.588
2013
34.882 -118.9194
2013
34.49149 -118.61004399999999
2013
34.90327 -118.93388
2013
38.774 -120.29799999999999
2013
34.071556 -117.048397
2013
40.00842 -120.76466599999999
2013
33.894055 -116.855019
2013
41.2415 -121.035
2013

35.85093 -120.81081
2016
32.9612 -116.7945
2016
5487.0 -120.841536
2016
39.33035 -121.78973
2016
39.79547 -121.71661999999999
2016
40.07846 -122.2071
2016
38.86804 -121.37704
2016
38.517771999999994 -121.103642
2016
35.70821 -120.82266000000001
2016
37.3228 -119.7738
2016
37.927613 -120.52883600000001
2016
38.8866 -122.04726000000001
2016
0.0 0.0
2016
36.499140000000004 -121.04993
2016
40.19611 -122.26736000000001
2016
0.0 0.0
2016
39.7947 -121.85700000000001
2016
36.92244 -119.39618999999999
2016
40.68192 -122.4096
2016
36.6543 -119.5327
2016
36.2833 -121.3306
2016
33.41639 -117.08828999999999
2016
39.0863 -122.7891
2016
38.40958 -122.43172
2016
35.77135 -120.251
2016
38.33475 -122.02564
2016
37.98171 -120.64241000000001
2016
0.0 0.0
2016
36.1934 -120.9293
2016
39.7299 -123.5044
2016
34.3183 -117.3189
2016
38.79549 -121.00299
2016
0.0 0.0
2016
39.43351 -121.6051
2016
39.20427 -121.05948000000001
2016
33.578720000000004 -117.77395
2016
38.5212 -120.8416
2016
36.099740000000004 -120.510

39.04786 -123.11971000000001
2018
40.896 -122.219
2018
41.38544 -121.04386000000001
2018
41.893332 -122.534655
2018
38.349000000000004 -119.929
2018
33.698879999999996 -117.52055
2018
40.633536 -120.86809099999999
2018
39.0674 -122.59848000000001
2018
0.0 0.0
2018
38.052054999999996 -120.94548200000001
2018
38.347395 -119.45186000000001
2018
37.393390000000004 -120.34206999999999
2018
34.22685 -118.93444
2018
0.0 0.0
2018
37.571 -119.118
2018
40.22027778 -122.18
2018
40.35068 -121.77866999999999
2018
41.079440000000005 -123.688056
2018
34.48229 -118.60858
2018
35.97896 -119.98329
2018
39.94355 -122.3571
2018
35.37444 -118.83556000000002
2018
36.6734 -118.21881
2018
38.69601 -122.44468
2018
39.64978 -122.58218000000001
2018
35.847778000000005 -120.343056
2018
38.431278000000006 -122.043747
2018
41.29363 -120.11461000000001
2018
37.40208 -118.50235
2018
40.99344 -121.52225
2018
36.85156 -120.77206000000001
2018
40.616251 -123.52018999999999
2018
40.50318 -122.42308
2018
38.21396 -121.942

In [ ]:

for city in cities:
    try:
        url = f'https://history.openweathermap.org/data/2.5/aggregated/year=?lat={lat}&lon={lon}&appid={weather_api_key}&units=metric'
        
        city_data = requests.get(url).json()
        print(f'Processing Record {record_counter} of Set {set_counter} | {city}')
        city_data_list.append([city,city_data['coord']['lat'],city_data['coord']['lon'],
                               city_data['main']['temp_max'],city_data['main']['humidity'],
                               city_data['clouds']['all'], city_data['wind']['speed'],
                               city_data['sys']['country'],city_data['dt']])
    except:
        print('City not found. Skipping...')
        print(city_data)

In [ ]:
# rds_connection_string = "postgres:Tgnome75@localhost:5432/customer_db"
# engine = create_engine(f'postgresql://{rds_connection_string}')
# new_customer_data_df.to_sql(name='customer_name', con=engine, if_exists='append', index=False)
# pd.read_sql_query('select * from customer_name', con=engine).head()